In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install -qqq streamlit --progress-bar off
!npm install -qqq localtunnel --progress-bar off

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
+ off@0.0.10
added 23 packages from 23 contributors and audited 23 packages in 2.308s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!pip3 install -qqq langchain --progress-bar off
!pip3 install -qqq llama-cpp-python --progress-bar off
!pip3 install -qqq sentence_transformers --progress-bar off
!pip3 install -qqq faiss-gpu --progress-bar off

!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.1-GGUF mistral-7b-instruct-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/comm

In [ ]:
!pip install PyPDF2
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.8 MB/s eta 0:00:00


In [ ]:
%%writefile rag_app.py

from langchain.llms import LlamaCpp
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
import streamlit as st

from langchain.text_splitter import RecursiveCharacterTextSplitter



from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# List of file paths for the five PDFs
pdf_files = [
    "/content/anime_wikipedia_content.pdf",
    "/content/myanimelist_content_report.pdf",
    "/content/Death_Note_Vol_01_of_12_Eng.pdf",

]

# Initialize an empty list to store the pages from all PDFs
all_pages = []

# Iterate over each PDF file
for file_path in pdf_files:
    # Load the PDF file using PyPDFLoader
    loader = PyPDFLoader(file_path)
    # Load and split the pages from the PDF
    pages = loader.load_and_split()
    # Extend the list of all pages with the pages from the current PDF
    all_pages.extend(pages)

# Now, the all_pages list contains all the pages from the five PDFs

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(all_pages)

# Now, the docs variable contains the text split into chunks



# llm
llm = LlamaCpp(model_path="/content/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
               max_tokens=2000,
               temperature=0.7,
               top_p=0.7,
               n_gpu_layers=-1,
               n_ctx=2048)

# splitting
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               #chunk_overlap=200)

#docs = text_splitter.split_documents(all_pages)

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

# vector database
vector_db = FAISS.from_documents(docs, embeddings)

memory = ConversationBufferMemory(memory_key='chat_history',
                                  return_messages=True,
                                  output_key='answer',  # Ensure this is 'answer'
                                  chat_history=[],
                                  max_history_size=1000)

# prompt

# Define prompt template using combined_pdf_text as context

template = """
<s> [INST]
You are a polite and professional question-answering AI assistant. You must provide a helpful response to the user.

In your response, PLEASE ALWAYS:
  (0) Be a detail-oriented reader: read the question and context and understand both before answering
  (1) Start your answer with a friendly tone, and reiterate the question so the user is sure you understood it
  (2) Provide a concise and to-the-point answer. Avoid lengthy explanations unless necessary.
  (3) If you can't find the answer, respond with an explanation, starting with: "I couldn't find the answer in the information I have access to".
  (4) Ensure your answer answers the question, is helpful, professional, and formatted to be easily readable.
[/INST]
[INST]
Answer the following question using the context provided.
The question is surrounded by the tags <q> </q>.
The context is surrounded by the tags <c> </c>.
<q>
{question}
</q>
<c>
{context}
</c>
[/INST]
</s>
[INST]
Model's Opinion or Feeling:
[INST]
If you're asking about my opinion or feeling, please note that I am not a human. I provide responses based on the information available to me and do not have personal opinions or feelings.
"""




prompt = PromptTemplate(template=template,
                        input_variables=["context", "question"])

# chain
# chain
chain = ConversationalRetrievalChain.from_llm(llm,
                                              retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
                                              memory=memory,
                                              return_source_documents=False,
                                              combine_docs_chain_kwargs={"prompt": prompt})





##### streamlit #####

st.title("Welcome Food Lovers")

# Initialise chat history
# Chat history saves the previous messages to be displayed
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("What do you wanna eat today! #ConversationalRetrievalChain"):

    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)

    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Begin spinner before answering question so it's there for the duration
    with st.spinner("Going down the rabbithole for answers..."):

        # send question to chain to get answer
        answer = chain(prompt)

        # extract answer from dictionary returned by chain
        response = answer["answer"]

        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(answer["answer"])

        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})

Writing rag_app.py


In [ ]:
%%writefile anime_app.py
from langchain.llms import LlamaCpp
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import streamlit as st

# Load anime content from PDFs
pdf_files = [
    "/content/anime_wikipedia_content.pdf",
    "/content/myanimelist_content_report.pdf",
    "/content/Death_Note_Vol_01_of_12_Eng.pdf",
]

all_pages = []
for file_path in pdf_files:
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    all_pages.extend(pages)

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(all_pages)

# Initialize llm
llm = LlamaCpp(model_path="/content/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
               max_tokens=2000,
               temperature=0.7,
               top_p=0.7,
               n_gpu_layers=-1,
               n_ctx=2048)

# Initialize embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model, cache_folder=embeddings_folder)

# Initialize vector database
vector_db = FAISS.from_documents(docs, embeddings)

# Initialize memory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer',
                                  chat_history=[], max_history_size=1000)

# Define prompt template
template = """
<s> [INST]
You are an anime expert chatbot. You must provide helpful responses to anime-related questions.

In your response, PLEASE ALWAYS:
  (0) Start your answer with a friendly tone.
  (1) Reiterate the question to confirm understanding.
  (2) Provide a concise and to-the-point answer.
  (3) If you can't find the answer, respond appropriately.

[INST]
Answer the following question using the context provided.
The question is surrounded by the tags <q> </q>.
The context is surrounded by the tags <c> </c>.
<q>
{question}
</q>
<c>
{context}
</c>
[/INST]
</s>
"""

# Initialize prompt
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Initialize chain
chain = ConversationalRetrievalChain.from_llm(llm, retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
                                              memory=memory, return_source_documents=False,
                                              combine_docs_chain_kwargs={"prompt": prompt})

# Streamlit UI
st.title("Anime Chatbot")
st.image("https://coisasdojapao.com/wp-content/uploads/2017/06/musicasdeanimesJap%C3%A3o.jpg", use_column_width=True)

# Chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history
for message in st.session_state.messages:
    with st.container():
        if message["role"] == "user":
            st.markdown(f"**User:** {message['content']}")
        elif message["role"] == "assistant":
            st.markdown(f"**Bot:** {message['content']}")

# User input
if user_input := st.text_input("You:"):
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.spinner("Bot is thinking..."):
        answer = chain(user_input)
        st.session_state.messages.append({"role": "assistant", "content": answer["answer"]})


Writing anime_app.py


In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [ ]:
%%writefile animeAI_app.py
#Open Ai and Langchain
from langchain.llms import LlamaCpp
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import streamlit as st
import openai

# Set your OpenAI API key
openai.api_key = 'sk-proj-hS10ucdJALR0AJPnJajKT3BlbkFJb1CtJbqN8pDkFDxArBQn'

# Load anime content from PDFs
pdf_files = [
    "/content/anime_wikipedia_content.pdf",
    "/content/myanimelist_content_report.pdf"
    #"/content/Death_Note_Vol_01_of_12_Eng.pdf",
]

all_pages = []
for file_path in pdf_files:
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    all_pages.extend(pages)

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(all_pages)

# Initialize llm
llm = LlamaCpp(model_path="/content/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
               max_tokens=2000,
               temperature=0.7,
               top_p=0.7,
               n_gpu_layers=-1,
               n_ctx=2048)

# Initialize embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model, cache_folder=embeddings_folder)

# Initialize vector database
vector_db = FAISS.from_documents(docs, embeddings)

# Initialize memory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer',
                                  chat_history=[], max_history_size=1000)

# Define prompt template
template = """
<s> [INST]
You are an anime expert chatbot. You must provide helpful responses to anime-related questions.

In your response, PLEASE ALWAYS:
  (0) Start your answer with a friendly tone.
  (1) Reiterate the question to confirm understanding.
  (2) Provide a concise and to-the-point answer.
  (3) If you can't find the answer, respond appropriately.

[INST]
Answer the following question using the context provided.
The question is surrounded by the tags <q> </q>.
The context is surrounded by the tags <c> </c>.
<q>
{question}
</q>
<c>
{context}
</c>
[/INST]
</s>
"""

# Initialize prompt
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Initialize chain
chain = ConversationalRetrievalChain.from_llm(llm, retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
                                              memory=memory, return_source_documents=False,
                                              combine_docs_chain_kwargs={"prompt": prompt})

# Streamlit UI
st.title("lets talk anime")
st.image("https://coisasdojapao.com/wp-content/uploads/2017/06/musicasdeanimesJap%C3%A3o.jpg", use_column_width=True)

# Chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history
for message in st.session_state.messages:
    with st.container():
        if message["role"] == "user":
            st.markdown(f"**User:** {message['content']}")
        elif message["role"] == "assistant":
            st.markdown(f"**Bot:** {message['content']}")

# User input
if user_input := st.text_input("You:"):
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.spinner("Netro is thinking..."):
        answer = chain(user_input)
        if answer["answer"]:  # Check if Langchain provided a response
            st.session_state.messages.append({"role": "assistant", "content": answer["answer"]})
        else:  # If Langchain didn't provide a response, use OpenAI
            openai_prompt = f"Question: {user_input}\nContext: Anime Chatbot\n"
            openai_response = openai.Completion.create(
                engine="text-davinci-002",  # Choose the engine based on your preferences
                prompt=openai_prompt,
                temperature=0.7,
                max_tokens=100
            ).choices[0].text.strip()
            st.session_state.messages.append({"role": "assistant", "content": openai_response})


Writing animeAI_app.py


In [ ]:
pip install langchain


In [ ]:
pip install --upgrade langchain


In [ ]:
from langchain.llms import LlamaGPT3

ImportError: cannot import name 'LlamaGPT3' from 'langchain.llms' (/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py)

In [ ]:
%%writefile animeAILLM_app.py
#Open Ai and Langchain instead on mistral
from langchain.llms import LlamaCpp
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import streamlit as st
import openai

# Set your OpenAI API key
openai.api_key = 'sk-proj-hS10ucdJALR0AJPnJajKT3BlbkFJb1CtJbqN8pDkFDxArBQn'

# Load anime content from PDFs
pdf_files = [
    "/content/anime_wikipedia_content.pdf",
    "/content/myanimelist_content_report.pdf"
    #"/content/Death_Note_Vol_01_of_12_Eng.pdf",
]

all_pages = []
for file_path in pdf_files:
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    all_pages.extend(pages)

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(all_pages)

# Set up Langchain with OpenAI's GPT-3
llm = LlamaGPT3(api_key='sk-proj-hS10ucdJALR0AJPnJajKT3BlbkFJb1CtJbqN8pDkFDxArBQn', engine='text-davinci-002')

# Initialize embeddings, vector database, memory, and prompt as before

# Initialize chain
chain = ConversationalRetrievalChain.from_llm(llm, retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
                                              memory=memory, return_source_documents=False,
                                              combine_docs_chain_kwargs={"prompt": prompt})


# Initialize embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model, cache_folder=embeddings_folder)

# Initialize vector database
vector_db = FAISS.from_documents(docs, embeddings)

# Initialize memory
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer',
                                  chat_history=[], max_history_size=1000)

# Define prompt template
template = """
<s> [INST]
You are an anime expert chatbot. You must provide helpful responses to anime-related questions.

In your response, PLEASE ALWAYS:
  (0) Start your answer with a friendly tone.
  (1) Reiterate the question to confirm understanding.
  (2) Provide a concise and to-the-point answer.
  (3) If you can't find the answer, respond appropriately.

[INST]
Answer the following question using the context provided.
The question is surrounded by the tags <q> </q>.
The context is surrounded by the tags <c> </c>.
<q>
{question}
</q>
<c>
{context}
</c>
[/INST]
</s>
"""

# Initialize prompt
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Initialize chain
chain = ConversationalRetrievalChain.from_llm(llm, retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
                                              memory=memory, return_source_documents=False,
                                              combine_docs_chain_kwargs={"prompt": prompt})

In [ ]:
pip install openai

In [ ]:
pip install openai --upgrade

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = "sk-proj-twv0jNl4WAZBOXCCoP1gT3BlbkFJX38wSiu4TXLbtetHisqp")

In [ ]:
%%writefile animeopenai_app.py
import streamlit as st
from openai import OpenAI

# Initialize OpenAI client with your API key
client = OpenAI(api_key="sk-proj-twv0jNl4WAZBOXCCoP1gT3BlbkFJX38wSiu4TXLbtetHisqp")

st.markdown(
    "<h1 style='text-align: center; color: #FF5733;'> The Anime Alchemist</h1>",
    unsafe_allow_html=True
      )
# Set page title and image
st.title("Let's Talk Anime !")
st.markdown(
    f"""
    <style>
        .center {{
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh; /* Set the height of the container to full viewport height */
            background-color: #f0f2f6; /* Background color of the container */
        }}
    </style>
    <div class="center">
        <img src="/content/drive/MyDrive/animedataset/isaac-netero-png-hunter-x-hunter-netero-png.png" style="width: 200px;">
    </div>
    """,
    unsafe_allow_html=True
)


# Chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history
for message in st.session_state.messages:
    with st.container():
        if message["role"] == "user":
            st.markdown(f"**You:** {message['content']}")
        elif message["role"] == "assistant":
            st.markdown(f"**Netro:** {message['content']}")

# User input
if user_input := st.text_input("You:"):
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.spinner("Netro is thinking..."):
        openai_prompt = f"Question: {user_input}\nContext: Anime Chatbot\n"
        openai_response = client.completions.create(
            model="gpt-3.5-turbo-instruct",  # Specify the correct model ID
            prompt=openai_prompt,
            stream=False  # Set stream to False
        ).choices[0].text.strip()
        st.session_state.messages.append({"role": "assistant", "content": openai_response})



Overwriting animeopenai_app.py


In [ ]:
%%writefile animeopenai_app.py
import streamlit as st
from openai import OpenAI

# Initialize OpenAI client with your API key
client = OpenAI(api_key="sk-proj-twv0jNl4WAZBOXCCoP1gT3BlbkFJX38wSiu4TXLbtetHisqp")

# Set page title and image
st.title("Let's Talk Anime !")
st.image("/content/drive/MyDrive/animedataset/isaac-netero-png-hunter-x-hunter-netero-png.png")#, width=350)


# Chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history
for message in st.session_state.messages:
    with st.container():
        if message["role"] == "user":
            st.markdown(f"**You:** {message['content']}")
        elif message["role"] == "assistant":
            st.markdown(f"**Netero:** {message['content']}", unsafe_allow_html=True)

# User input
user_input = st.text_input("You:")
if user_input:
    st.session_state.messages.append({"role": "user", "content": user_input})

openai_response = None

if user_input:
    with st.spinner("Netero is thinking..."):
        openai_prompt = f"Question: {user_input}\nContext: Anime Chatbot\n"
        openai_response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=openai_prompt,
            max_tokens=200,
            stream=False
        ).choices[0].text.strip()

# Display assistant's response immediately
if openai_response:
    st.session_state.messages.append({"role": "assistant", "content": openai_response})



In [ ]:
#Working
%%writefile animeopenai_app.py
import streamlit as st
from openai import OpenAI

# Initialize OpenAI client with your API key
client = OpenAI(api_key="sk-proj-twv0jNl4WAZBOXCCoP1gT3BlbkFJX38wSiu4TXLbtetHisqp")

# Set page title and image
st.title("Let's Talk Anime !")
st.image("/content/drive/MyDrive/animedataset/isaac-netero-png-hunter-x-hunter-netero-png.png")

# Chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history
for message in st.session_state.messages:
    with st.container():
        if message["role"] == "user":
            st.markdown(f"**You:** {message['content']}")
        elif message["role"] == "assistant":
            st.markdown(f"**Netero:** <span style='color:red'>{message['content']}</span>", unsafe_allow_html=True)

# User input
user_input = st.text_input("You:", key="user_input")
if user_input:
    st.session_state.messages.append({"role": "user", "content": user_input})

    openai_prompt = f"Question: {user_input}\nContext: Anime Chatbot\n"
    openai_response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=openai_prompt,
        max_tokens=200,
        stream=False
    ).choices[0].text.strip()

    # Display assistant's response immediately
    if openai_response:
        st.session_state.messages.append({"role": "assistant", "content": openai_response})


Overwriting animeopenai_app.py


In [ ]:
%%writefile animeopenai_app.py
import streamlit as st
from openai import OpenAI

# Initialize OpenAI client with your API key
client = OpenAI(api_key="sk-proj-twv0jNl4WAZBOXCCoP1gT3BlbkFJX38wSiu4TXLbtetHisqp")

st.markdown(
    "<h1 style='text-align: center; color: #FF5733;'> The Anime Alchemist</h1>",
    unsafe_allow_html=True
      )
# Set page title
st.title("Let's Talk Anime !")

# Create a layout with two columns
col1, col2 = st.columns([4, 2])  # Adjust the ratio as needed

# Chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# User input
user_input = col1.text_input("You:", key="user_input")
if user_input:
    st.session_state.messages.append({"role": "user", "content": user_input})

    openai_prompt = f"Question: {user_input}\nContext: Anime Chatbot\n"
    openai_response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=openai_prompt,
        max_tokens=200,
        stream=False
    ).choices[0].text.strip()

    # Display assistant's response immediately
    if openai_response:
        st.session_state.messages.append({"role": "assistant", "content": openai_response})

# Display chat messages from history
for message in st.session_state.messages:
    if message["role"] == "user":
        col1.markdown(f"**You:** {message['content']}")
    elif message["role"] == "assistant":
        col1.markdown(f"**Netero:** <span style='color:red'>{message['content']}</span>", unsafe_allow_html=True)

# Display image in the second column
col2.image("/content/drive/MyDrive/animedataset/isaac-netero-png-hunter-x-hunter-netero-png.png", width=400)


Overwriting animeopenai_app.py


In [ ]:
!streamlit run animeopenai_app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.148.193.166
npx: installed 22 in 2.91s
your url is: https://calm-facts-juggle.loca.lt
